This is an example of a RAG search engine powered by Openai and Pinecone

In [1]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Load environment variables
load_dotenv()

# Set up Pinecone API key and initialize
pinecone_api_key = os.getenv("PINECONE_API_KEY")
if not pinecone_api_key:
    raise ValueError("PINECONE_API_KEY is not set in the .env file")

pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI()

# Connect to the index
index_name = 'idea-index'
namespace = os.getenv("PINECONE_NAMESPACE")
index = pc.Index(index_name)

# Define functions
def get_embeddings(text, model="text-embedding-3-large"):
    """Generate embeddings using OpenAI."""
    response = client.embeddings.create(
        model=model,
        input=text
    )
    return response.data[0].embedding

def get_chat_completion(prompt, model="gpt-4o"):
    """Generate chat completion using OpenAI."""
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an assistant for an innovation platform. Your task is to help users by providing answers or suggestions based on ideas submitted to the platform. Use the following retrieved ideas as context to answer the user's query effectively."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content


def query_pinecone(query_text, index, top_k=5):
    """Query Pinecone index with a text input."""
    query_embedding = get_embeddings(query_text)
    response = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace
    )
    
    prompt = (
        f"Use the following ideas to answer the user's question:\n"
        f"{response}\n"
        f"### User's Question:\n"
        f"{query_text}\n\n"
        f"### Instructions:\n"
        f"- Focus your response on the user's query.\n"
        f"- Use the provided ideas to give specific, actionable answers.\n"
        f"- If no ideas seem directly relevant, provide general advice or suggest next steps for the user to refine their question.\n\n"
        f"### Response:"
    )

    answer = get_chat_completion(prompt, model="gpt-4o")

    return answer

# Interactive Cells

# Query Example
query_text = input("\nEnter a query to search Pinecone: ")
results = query_pinecone(query_text, index)
print(f"Query Results: {results}")


Query Results: Here are some ideas related to hackathons that have been submitted to the platform:

1. **Updating Hackathon Support Pages**:
   - **Submitted by Michael Lutjen on 09/22/2023**: This idea focuses on updating the Hackathon support pages to reflect new branding, including name changes from "Hack" to "Hackathon" and updating orange icons to a red design. This has been implemented, indicating the changes have been made.

2. **Updating References in Product**:
   - **Submitted by Michael Lutjen on 09/22/2023**: Similar to the support pages update, this idea suggests reflecting branding updates in the product itself by changing references from "Hack" to "Hackathon" and updating color schemes. This idea has also been implemented.

3. **Renaming Steps in Hackathon Pipeline**:
   - **Submitted by Veronika Vacek on 06/24/2024**: Proposes renaming the first step in the hackathon process from "Initial Submission" to "Incubate" to better align with the hackathon's goal of building ou